# Case Study by Linguistic Features Visualization


This notebook is to visualize important linguistic features in the NPS prediction model and perform the case study to provide suggestions for unpopular courses.

## 1. Import Packages

In [1]:
import pandas as pd
import numpy as np
import zipfile
import os
import collections
from collections import defaultdict
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
! pip install pysrt
import matplotlib.pyplot as plt
!pip install textatistic
from textatistic import Textatistic
from matplotlib.pyplot import figure
import pysrt
! pip install sentence_transformers
from sentence_transformers import SentenceTransformer
import spacy
from scipy import spatial

You should consider upgrading via the '/Users/qichao/opt/miniconda3/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/qichao/opt/miniconda3/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/qichao/opt/miniconda3/bin/python3 -m pip install --upgrade pip' command.


## 2. Import Data 

Import the survey data (`'EndOfCourseSurvey.csv'`) and the course transcript data from zip file (`'SRT files-20220429T000407Z-001.zip'`).

### 2.1. Survey Data 

In [3]:
df = pd.read_csv('EndOfCourseSurvey.csv')
df.head()

/Users/qichao/opt/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,FactId,DateCompleted,Question_1,Response_Q1,Question_2,Response_Q2,Question_3,Response_Q3,Question_4,Response_Q4,...,Response_Q5,CustomerID,CourseName,Instructor,Program,Requirement,Level,VideoLenght,Price,SurveyType
0,142055,2021-07-07 12:28:00.0000000,How likely is it that you would recommend this...,9,This course covered the topics and content tha...,Agree,"The course materials were engaging, clear, and...",Agree,The instructor was effective in delivering the...,Agree,...,Did Not Respond,198151,Building a 3 Statement Financial Model,Tim Vipond,FMVA,Core,3,2-3 hrs,197.0,NewSurvey
1,142056,2021-08-12 14:42:00.0000000,How likely is it that you would recommend this...,9,This course covered the topics and content tha...,Agree,"The course materials were engaging, clear, and...",Agree,The instructor was effective in delivering the...,Agree,...,Did Not Respond,152550,Building a 3 Statement Financial Model,Tim Vipond,FMVA,Core,3,2-3 hrs,197.0,NewSurvey
2,142057,2021-08-18 03:46:00.0000000,How likely is it that you would recommend this...,10,This course covered the topics and content tha...,Strongly Agree,"The course materials were engaging, clear, and...",Strongly Agree,The instructor was effective in delivering the...,Strongly Agree,...,Did Not Respond,191851,Building a 3 Statement Financial Model,Tim Vipond,FMVA,Core,3,2-3 hrs,197.0,NewSurvey
3,142058,2021-08-22 16:34:00.0000000,How likely is it that you would recommend this...,10,This course covered the topics and content tha...,Strongly Agree,"The course materials were engaging, clear, and...",Strongly Agree,The instructor was effective in delivering the...,Strongly Agree,...,Did Not Respond,263617,Building a 3 Statement Financial Model,Tim Vipond,FMVA,Core,3,2-3 hrs,197.0,NewSurvey
4,142059,2021-08-25 01:48:00.0000000,How likely is it that you would recommend this...,10,This course covered the topics and content tha...,Strongly Agree,"The course materials were engaging, clear, and...",Strongly Agree,The instructor was effective in delivering the...,Strongly Agree,...,Did Not Respond,305095,Building a 3 Statement Financial Model,Tim Vipond,FMVA,Core,3,2-3 hrs,197.0,NewSurvey


### 2.2. Transcript Data 

#### 2.2.1. Feature Extraction

The pipeline below is to import the transcript text data and extract the important linguistic features to store them in dictionaries.
The `try-except` syntax is to skip the un-decodable srt files which are proven to be empty.

- The features are stored as the following format:

{`'course_name_1'`: {`'video_1'`: `feature_value`,
                    `'video_2'`: `feature_value`,
                    ...
                    ...}
                  
`'course_name_2'`: {`'video_1'`: `feature_value`,
                    `'video_2'`: `feature_value`,
                    ...
                    ...}

...

...


}

In [4]:
zip_file = zipfile.ZipFile('SRT files-20220429T000407Z-001.zip', 'r')
files = zip_file.infolist()

duration_dict = defaultdict(dict)
text_dict = defaultdict(dict)
word_count = defaultdict(dict)
word_per_second = defaultdict(dict)
flesch_score = defaultdict(dict)

for i in range(len(zip_file.infolist())):

    try:
        chapter_text = ''
        chapter_duration = pysrt.srttime.SubRipTime()
        #print(chapter_duration)
        filename = files[i].filename
        path_to_extracted_file = zip_file.extract(filename)
        course_name = filename.split('/')[1]
        #print(course_name)
        section_name = filename
        #print(section_name)
        subs = pysrt.open(path_to_extracted_file)
        

        for j in range(len(subs)):
            sub = subs[j]
            course_text = sub.text + ' '
            duration = sub.duration
            chapter_duration += duration
            chapter_text += course_text
            chapter_seconds = chapter_duration.milliseconds/1000 + chapter_duration.seconds + chapter_duration.minutes*60 + chapter_duration.hours*3600
            
        duration_dict[course_name][section_name] = chapter_seconds
        text_dict[course_name][section_name] = chapter_text
        
        w_count = len(tokenizer.tokenize(chapter_text))
        word_count[course_name][section_name] = w_count
        word_per_second[course_name][section_name] = w_count/chapter_seconds
        flesch = Textatistic(chapter_text).scores['flesch_score']
        flesch_score[course_name][section_name] = flesch
    
    except:
        pass



Have a look at the duration feature of course `'Rocky Mountain Holdings Limited Case Study'`:

In [9]:
duration_dict['Rocky Mountain Holdings Limited Case Study']

{'SRT files/Rocky Mountain Holdings Limited Case Study/RockyMountainLtd_11_Analysis Tab.srt': 74.92,
 'SRT files/Rocky Mountain Holdings Limited Case Study/RockyMountainLtd_19_Summary.srt': 63.92,
 'SRT files/Rocky Mountain Holdings Limited Case Study/RockyMountainLtd_14_Debrief - Analysis Tab Interpretation.srt': 186.32,
 'SRT files/Rocky Mountain Holdings Limited Case Study/RockyMountainLtd_16_Debrief - Risk Rating and Pricing.srt': 170.52,
 'SRT files/Rocky Mountain Holdings Limited Case Study/RockyMountainLtd_10_Debreif - Rent Roll Tab.srt': 135.645,
 'SRT files/Rocky Mountain Holdings Limited Case Study/RockyMountainLtd_09_Rent Roll Tab.srt': 37.48,
 'SRT files/Rocky Mountain Holdings Limited Case Study/RockyMountainLtd_13_Debrief - Analysis Tab Expenses.srt': 185.14,
 'SRT files/Rocky Mountain Holdings Limited Case Study/RockyMountainLtd_18_Debrief - Discussion Paper.srt': 354.061,
 'SRT files/Rocky Mountain Holdings Limited Case Study/RockyMountainLtd_06_Debrief - Timeline.srt':

#### 2.2.1. Data Transformation

The video name are not well sorted. the function below is to sort the videos in each course by their name:

In [10]:
def sort_dicts_in_dict(dictionary):

    '''sort the dictionaries in a dictionary by the keys'''
    
    for (course_name, course_transcript) in dictionary.items():
        sorted_course_transcript = sorted(course_transcript.items())
        dictionary[course_name] = list(dict(sorted_course_transcript).values())
        
    return dictionary

Apply the function to the duration feature dictionary:

In [11]:
duration_dict = sort_dicts_in_dict(duration_dict)

In [12]:
duration_dict['Rocky Mountain Holdings Limited Case Study']

[94.03999999999999,
 99.82,
 252.66,
 297.54,
 378.48,
 145.41,
 291.421,
 90.98,
 37.48,
 135.645,
 74.92,
 93.27000000000001,
 185.14,
 186.32,
 115.36,
 170.52,
 54.36,
 354.061,
 63.92]

Now the feature values are well sorted from video 1 to video 19. We don't need the video name as the key any more. The feature values are stored in a list to be further transformed into columns in a data frame.

We can apply the function to the rest of the dictionaries.

In [14]:
text_dict = sort_dicts_in_dict(text_dict)
word_count = sort_dicts_in_dict(word_count)
word_per_second = sort_dicts_in_dict(word_per_second)
flesch_score = sort_dicts_in_dict(flesch_score)

The dictionaries can be converted into data frames for merging and visualization.

In [15]:
duration_df = pd.DataFrame(duration_dict.items(), columns=['CourseName', 'Duration'])
transcript_df = pd.DataFrame(text_dict.items(), columns=['CourseName', 'Transcript'])
word_count_df = pd.DataFrame(word_count.items(), columns=['CourseName', 'word_count'])
word_per_second_df = pd.DataFrame(word_per_second.items(), columns=['CourseName', 'word_per_second'])
flesch_score_df = pd.DataFrame(flesch_score.items(), columns=['CourseName', 'flesch_score'])

In [16]:
duration_df

,CourseName,Duration
0,Rocky Mountain Holdings Limited Case Study,"[94.03999999999999, 99.82, 252.66, 297.54, 378..."
1,Refinitiv Workspace Fundamentals,"[47.03, 70.26, 84.17, 103.38, 110.460000000000..."
2,Vertical IQ Fundamentals,"[95.84, 152.127, 42.04, 196.77, 118.08, 196.10..."
3,Scenario _ Sensitivity Analysis in Excel,"[101.22, 51.39, 90.66, 61.4, 106.5099999999999..."
4,Statistics Fundamentals,"[110.50999999999999, 103.74000000000001, 51.96..."
...,...,...
127,Credit Administration and Documentation,"[100.22999999999999, 105.62299999999999, 76.82..."
128,"Equity, FX, and Rate Futures","[12.42, 104.47999999999999, 107.779, 116.24000..."
129,Commercial Banking - Debt Modeling,"[63.18, 118.93, 160.32, 156.76, 91.58, 70.11, ..."
130,Loan Covenants,"[125.96, 120.79, 75.28, 104.328, 93.16, 142.07..."


<br><br><br><br><br>

**For collaboration purpose, The data transformation and processing are performed in google Colab. So the transformed data can be used directly in the file.**

# Import data

The `chapter_nps_df_120` data frame contains 120 validated courses data. Each row represent one video of a course.

In [21]:
import pandas as pd
import altair as alt

In [22]:
chapter_nps_df_120 = pd.read_csv("chapter_nps_df_120.csv")

In [23]:
chapter_nps_df_120.head()

,Unnamed: 0,CourseName,Transcript,Response_Q1,Duration,ChapterName,Chapter_count,word_count,sent_count,avg_sentence_length,...,Program_x,Requirement_x,Level_x,VideoLenght_x,Price_x,Program_y,Requirement_y,Level_y,VideoLenght_y,Price_y
0,0,Rocky Mountain Holdings Limited Case Study,"Hi. I'm Kyle Peterdy, VP, commercial credit an...",9.14554,94.04,SRT files/Rocky Mountain Holdings Limited Case...,19,262,12,21.833333,...,CRE,Case Study,4,Under 1 hr,147.0,CRE,Case Study,4,Under 1 hr,147.0
1,1,Rocky Mountain Holdings Limited Case Study,Please ensure that your due diligence package ...,9.14554,99.82,SRT files/Rocky Mountain Holdings Limited Case...,19,276,14,19.714286,...,CRE,Case Study,4,Under 1 hr,147.0,CRE,Case Study,4,Under 1 hr,147.0
2,2,Rocky Mountain Holdings Limited Case Study,"Before we get into the due diligence package, ...",9.14554,252.66,SRT files/Rocky Mountain Holdings Limited Case...,19,660,29,22.758621,...,CRE,Case Study,4,Under 1 hr,147.0,CRE,Case Study,4,Under 1 hr,147.0
3,3,Rocky Mountain Holdings Limited Case Study,Many of the line items in a purchase and sale ...,9.14554,297.54,SRT files/Rocky Mountain Holdings Limited Case...,19,818,44,18.590909,...,CRE,Case Study,4,Under 1 hr,147.0,CRE,Case Study,4,Under 1 hr,147.0
4,4,Rocky Mountain Holdings Limited Case Study,"First, let's add to our timeline. The appraisa...",9.14554,378.48,SRT files/Rocky Mountain Holdings Limited Case...,19,1011,49,20.632653,...,CRE,Case Study,4,Under 1 hr,147.0,CRE,Case Study,4,Under 1 hr,147.0


These are the columns of the data frame. they are used to train the Machine Learning models in another file.

Here we only used the `word_per_second`(speech speed), `flesch_reading_ease` and `avg_coherence` for visualization.

In [25]:
chapter_nps_df_120.columns

Index(['Unnamed: 0', 'CourseName', 'Transcript', 'Response_Q1', 'Duration',
       'ChapterName', 'Chapter_count', 'word_count', 'sent_count',
       'avg_sentence_length', 'avg_syllables_per_word', 'difficult_words',
       'verb_count', 'nnp_count', 'nn_count', 'adverb_count', 'ner_count',
       'flesch_reading_ease', 'gunning_fog', 'smog_index',
       'dale_chall_readability_score', 'avg_coherence', 'word_per_second',
       'NPS_score', 'Program_x', 'Requirement_x', 'Level_x', 'VideoLenght_x',
       'Price_x', 'Program_y', 'Requirement_y', 'Level_y', 'VideoLenght_y',
       'Price_y'],
      dtype='object')

These 3 columns are added for visualization legends:

In [26]:
chapter_nps_df_120["speed"] = "Speed (words/sec)"
chapter_nps_df_120["coherence"] = "Coherence"
chapter_nps_df_120["readability"] = "Readability"

The function below is for the final visualization. We can modify the 

In [34]:
def viz_lines(CourseName):
    '''For a input course name,
    visualize the overlapping line graphs 
    of speed, readability and coherence by pairs'''
    course_df = chapter_nps_df_120[chapter_nps_df_120["CourseName"] == CourseName]
    course_df['Section'] = range(1, len(course_df)+1)
    #Title = str(CourseName) + "(NPS:{})".format(course_df['Response_Q1'][0])

    speed = alt.Chart(course_df, title=CourseName).mark_line(color='red').encode(
        x=alt.X('Section', axis=alt.Axis(tickCount=course_df.shape[0], title="Section")),
        y=alt.Y('word_per_second', axis=alt.Axis(title='', titleColor='red')),
        opacity=alt.Opacity('speed', legend=alt.Legend(title=""))
    ).properties(
    width=1000
    )
    

    readability = alt.Chart(course_df).mark_line(color='green').encode(
        x=alt.X('Section', axis=alt.Axis(title="Section")),
        y=alt.Y('flesch_reading_ease', axis=alt.Axis(title='', titleColor='green')),
        opacity=alt.Opacity('readability', legend=alt.Legend(title=""))
    ).properties(
        width=1000
    )

    
    coherence = alt.Chart(course_df).mark_line(color='blue').encode(
        x=alt.X('Section', axis=alt.Axis(title="Section")),
        y=alt.Y('avg_coherence', axis=alt.Axis(title='', titleColor='blue',ticks=False,labels=False)),
        opacity=alt.Opacity('coherence', legend=alt.Legend(title=""))
    ).properties(
        width=1000
    )
    

    
    graph = alt.layer(speed, coherence, readability).resolve_scale(
        y = 'independent',
        opacity='independent'
    )

    return graph 


In [35]:
viz_lines("Introduction to ESG")


/var/folders/3z/vln54yld7pl4fbx7dy7pfg580000gp/T/ipykernel_74166/3598757986.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  course_df['Section'] = range(1, len(course_df)+1)


alt.LayerChart(...)

In [36]:
viz_lines("ESG Disclosure")
# Lowest NPS score

/var/folders/3z/vln54yld7pl4fbx7dy7pfg580000gp/T/ipykernel_74166/3598757986.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  course_df['Section'] = range(1, len(course_df)+1)


alt.LayerChart(...)

In [37]:
viz_lines("Case Study - RockCrusher Rentals")
# Highest NPS score 9.25

/var/folders/3z/vln54yld7pl4fbx7dy7pfg580000gp/T/ipykernel_74166/3598757986.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  course_df['Section'] = range(1, len(course_df)+1)


alt.LayerChart(...)

<br><br><br><br><br><br><br><br>